## Notebook that demonstrates requesting ERA5 data from the ECWMF Climate Data Store, and plotting a basic map of the low-level jet

### If you haven't obtained data from ECWMF CDS before, here are some instructions to make an account and set things up on your computer: https://cds.climate.copernicus.eu/api-how-to

#### Imports

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.gridspec as gridspec
import matplotlib.pylab as plt
import metpy.calc as mpcalc
from metpy.units import units
from metpy.plots import ctables
import numpy as np
import scipy.ndimage as ndimage
import xarray as xr
import matplotlib.colors as mcolors
import os
import pandas as pd

### you may not yet have these packages installed in your environment, so may need to install them now.
### For example, after activating your ats641_2022 environment, 'conda install -c conda-forge cdsapi fsspec'
import cdsapi
import fsspec

### Define the date to plot

In [ ]:
date = pd.Timestamp(2015,6,3)

### get data from the ECMWF API. We'll get basic fields at 850 and 900 mb. Right now this is set to just get all 24 hours from the day before the time specified through the day after, but this can be adjusted. (If your date is at the beginning or end of month, you may need to adjust manually.) 


### Usually the data will be obtained very quickly, but sometimes there will be a wait in the queue if the ECMWF system is busy.


In [ ]:
date_m1 = date - pd.Timedelta(days=1)
date_p1 = date + pd.Timedelta(days=1)

## get the data from ECMWF API
## instead of downloading it, read it directly
## based on: https://nbviewer.jupyter.org/github/pangeo-data/pangeo-era5/blob/master/lazy_loading_expts.ipynb

#######pressure levels
c1 = cdsapi.Client()
name = 'reanalysis-era5-pressure-levels'
request = {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            'geopotential', 'relative_humidity', 'temperature',
            'u_component_of_wind', 'v_component_of_wind',
        ],
        'pressure_level': [
            '850','900',
        ],
        'year': date.year,
        'month': date.month,
        'day': [date_m1.day, date.day, date_p1.day],
        'time': [
                '00:00', '01:00', '02:00',
                '03:00', '04:00', '05:00',
                '06:00', '07:00', '08:00',
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ],
        'area'          : [60.,230, 20., 290.] #N,W,S,E
}
r1 = c1.retrieve(name, request, None)

with fsspec.open(r1.location) as f1:
    data = xr.open_dataset(f1, engine='scipy')
    
    
#######also get the surface pressure
c2 = cdsapi.Client()
name = 'reanalysis-era5-single-levels'
request = {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            'surface_pressure',
        ],
        'year': date.year,
        'month': date.month,
        'day': [date_m1.day, date.day, date_p1.day],
        'time': [
                '00:00', '01:00', '02:00',
                '03:00', '04:00', '05:00',
                '06:00', '07:00', '08:00',
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ],
        'area'          : [60.,230, 20., 290.] #N,W,S,E
}
r2 = c2.retrieve(name, request, None)

with fsspec.open(r2.location) as f2:
    data_sfc = xr.open_dataset(f2, engine='scipy')
    

Check out what's in the data

In [ ]:
#data
data_sfc

### Do some smoothing of the 900-hPa data

In [ ]:
### ERA5 data comes as geopotential, convert it to geopotential height
hght_900 = (data['z'].sel(level=900))/9.81
hght_900 = mpcalc.smooth_n_point(hght_900, 9) ## more aggressive smoothing of height
uwnd_900 = mpcalc.smooth_n_point(data['u'].sel(level=900), 5)
vwnd_900 = mpcalc.smooth_n_point(data['v'].sel(level=900), 5)
tmpk_900 = mpcalc.smooth_n_point(data['t'].sel(level=900), 5)

tmpc_900 = tmpk_900.metpy.convert_units("degC")
wspd_900 = mpcalc.wind_speed(uwnd_900,vwnd_900)

In [ ]:
wspd_900

### And plot the map (looping over hours)

In [ ]:
n_hours = uwnd_900.shape[0]

# Set Projection of Data
datacrs = ccrs.PlateCarree()

# Set Projection of Plot
plotcrs = ccrs.LambertConformal(central_latitude=45, central_longitude=-100)

#for tt in range(0,n_hours):  ## loop over times
for tt in range(0,1):
    
    vtime = pd.Timestamp(wspd_900[tt,:,:].time.values)
    print(vtime)

    fig = plt.figure(1, figsize=(20,16))
    gs = gridspec.GridSpec(2, 1, height_ratios=[1, .02], bottom=.07, top=.99,
                           hspace=0.01, wspace=0.01)
    ax = plt.subplot(gs[0], projection=plotcrs)

    # Plot Titles
    plt.title(r'GFS analysis 900-hPa heights and wind speed',
              loc='left', fontsize=18)
    plt.title(vtime.strftime("%H%M UTC %d %b %Y"), loc='right', fontsize=18)

    # Plot Background
    #ax.set_extent([217.5, 290., 18., 66.])
    ax.set_extent([242., 281., 21., 51.])
    ax.coastlines('50m', edgecolor='black', linewidth=0.75, zorder=10)
    ax.add_feature(cfeature.STATES, linewidth=.5, zorder=11)

    # Plot Height Contours
    clev900 = np.arange(0,3001,30)
    cs = ax.contour(data.longitude, data.latitude, hght_900[tt,:,:], clev900, colors='black', linewidths=2,
                    linestyles='solid', transform=ccrs.PlateCarree())
    plt.clabel(cs, fontsize=12, inline=1, inline_spacing=10, fmt='%i',
               rightside_up=True, use_clabeltext=True)

    # Plot colorfill and dashed contours of 900-hPa wind speed
    clevs_900_wspd = np.arange(6,25,1)
    cf = ax.contourf(data.longitude, data.latitude, wspd_900[tt,:,:], clevs_900_wspd, alpha=0.9,
                     cmap=plt.cm.Oranges, extend='max', transform=datacrs)
    cb = plt.colorbar(cf, orientation='horizontal', pad=0.01, aspect=50, shrink=0.9)
    cb.set_label('wind speed (m/s)', fontsize=14)
    cb.ax.tick_params(labelsize=14)

    # Plot Wind Barbs
    # Transform Vectors and plot wind barbs.
    wind_slice = slice(None, None, 6)
    ax.barbs(uwnd_900.longitude[wind_slice], uwnd_900.latitude[wind_slice], uwnd_900[tt,wind_slice,wind_slice].metpy.convert_units('kt').values, 
             vwnd_900[tt,wind_slice,wind_slice].metpy.convert_units('kt').values, 
             length=8, linewidth=1,pivot='middle', transform=ccrs.PlateCarree())
    
    ### and mask out the area that's underground
    cf2 = ax.contourf(data.longitude,data.latitude,data_sfc.sp[tt,:,:]/100., [0,900.],
                     colors=['lightgray'], zorder=4, transform=ccrs.PlateCarree())

    #outfile="900wspd_era5_"+vtime.strftime('%Y%m%d')+"_"+vtime.strftime('%H%M')+".png"
    #fig.savefig(outfile, dpi=255, bbox_inches='tight', transparent=False, facecolor='white')
    #os.system("mv "+outfile+" "+case_date+"/.")
    plt.show()

    plt.close('all')